<a href="https://colab.research.google.com/github/laurence-lin/tempotron/blob/master/tempotron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import csv
import os
import time


import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf

print('Library imported.')



Library imported.


In [9]:

# Input value encoding: Gaussian receptive field
gauss_neuron = 12  # each input value is encoded by 12 gaussian neurons
center = np.ones((gauss_neuron, 1))
width = 1/15

# let the value range equal in each feautres, thus gaussian encoding center for each feature is the same
for i in range(len(center)): # let X_max=1, x_min=0. Center for each feature=i/(n - 1)
  center[i] = (2*i-3)/20  # Center of gaussian shift from left to right

x = np.arange(0, 1, 0.0001) # input variable resolution in gaussian receptive field

num_features = 4
gauss_recpt_field = np.zeros((gauss_neuron, len(x))) # gaussian receptive field for input stimulate response
spike_time = np.zeros((gauss_neuron, num_features)) # 48 gaussian encoding neurons for 4 input features

# Define gaussian receptive field for input variables entries
for i in range(gauss_neuron):
  gauss_recpt_field[i, :] = np.exp( -(x-center[i])**2/(2*width*width) ) # different shift for each gaussian function


def gauss_response(inputs):
  # input: shape [1, features] 
  # output: shape [gaussian neurons*features] spiking time
  
  for i in range(num_features):
    # for each input[i]
    for j in range(gauss_neuron):
      spike_time[j, i] = gauss_recpt_field[j, inputs[i]] # input feature enter the gaussian entry, recieve response

  spikes = []
  for i in range(spike_time.shape[1]):
    spikes.extend(spike_time[:, i])

  return np.array(spikes)




In [3]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)

encoder = LabelEncoder()
df.iloc[:, 4] = encoder.fit_transform(df.iloc[:, 4])
print(df.head())

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# shuffle data
df = df.sample(frac=1).reset_index(drop = True)
print(df.head())

X = df.iloc[:, 0:4]
y = df.iloc[:, -1]

gauss_neurons = gauss_neuron*num_features

# Normalization
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# There are 10000 entries in each gaussian function
X = (X*10000).astype(int) # meet the index of gaussian receptive field, drop the fourth decimal
X[X == 10000] = 9999 # convert all input value indexing to gaussian matrix entry
input_spike = np.zeros((X.shape[0], gauss_neurons)) # spike time in [0, 1]

for i in range(X.shape[0]):
  input_spike[i, :] = gauss_response(X[i, :])

input_spike[input_spike < 0.1] = 0  # set activate value < 0.1 to zero
input_spike = np.around(100*(1 - input_spike)) # convert actiate value to 0~100 timing, round the decimals
#Adjust t = 0 firing to t = 1
input_spike[input_spike == 0] = 1


x_train, x_valid, y_train, y_valid = train_test_split(input_spike, y, test_size = 0.2)

     0    1    2    3  4
0  5.1  3.5  1.4  0.2  0
1  4.9  3.0  1.4  0.2  0
2  4.7  3.2  1.3  0.2  0
3  4.6  3.1  1.5  0.2  0
4  5.0  3.6  1.4  0.2  0


In [17]:
# Start training

def heavyside(t, t_i):
  if (t - t_i) >= 0:
    return 1
  elif (t - t_i) < 0:
    return 0

def K_res(t, t_i):
  time = np.arange(0, 100, 1)
  tou_m = 15
  tou_s = tou_m/4
  V_0 = 1/( (np.exp(-(time)/tou_m) - np.exp(-(time)/tou_s)).max() ) #normalization factor: divide the K() response by maximum value
  response = V_0*( np.exp(-(time)/tou_m) - np.exp(-(time).tou_s) )*heavyside(t, t_i)
  return response


iterations = 50
num_class = 3
V = np.zeros((num_class, 100)) # potential of each output neuron
V_rest = 0
weight = np.random.random((gauss_neurons, num_class)) # connections btw input neuron and output neuron(here use single synaptic for simplicity)
threshold = 1 # set threshold: try and error
T = 100    # time encoding window
firing_time = np.zeros((1, gauss_neurons)) # time-to-first-spike for all gaussian neurons
Response = np.zeros((1, gauss_neurons))  # K(t-t_i) response for each gaussian neurons
lr = 0.005
tou_m = 15
time = np.arange(1, T+1)
accuracy = np.zeros((1, iterations))

def train():  

   for iterate in range(iterations):

      # Check the accuracy after training each classifier
      correct = 0 
      val_predict = np.zeros((y_valid.shape))
      for valid_sample in range(y_valid.shape[0]):
        t_max = T*np.ones((1, num_class)) # default firing time: all neuron's maximum firing time at 100ms
        max_state = np.zeros((1, num_class)) # max potential state of each output neuron
        # Go through a time window, observe the firing history
        for t in range(T):
          for neuron in range(gauss_neurons):
            Response[neuron] = K_res(t, y_valid[valid_sample, neuron])

          #calculate PSP for output neuron 
          for out_id in range(num_class):
            V[out_id, t] = Response*weight[:, out_id] + V_rest

        # Find the firing timing t_max
        for out_id in range(num_class):
          fire_time = np.where((V > threshold)[out_id, :])[0]
          if len(fire_time) > 0: # set firing time between 1 to 100
            max_state[0, out_id] = fire_time[0] + 1
            t_max[out_id] = fire_time[0] + 1
          else:
            max_state[0, out_id] = T
          
          if max_state[0, out_id] < T: # if output neuron fires
            V[out_id, t_max[out_id]:] = V[out_id, t_max[out_id]]*np.exp((time[t_max[out_id]:]-time[t_max[out_id]])/tou_m)
        # We only need t_max for prediction?
        pred_class = np.argmin(t_max)
        val_predict[valid_id] = pred_class

      acc = accuracy_score(y_valid, val_predict)
      print('Validation accuaracy at iteration {}: {}'.format(str(iterate+1), acc))

      # Training
      for sample in range(x_train.shape[0]):
        t_max = 100*np.ones((1, num_class))
        max_state = np.zeros((1, num_class))
        for t in range(T):
          for neuron in range(gauss_neurons):
            Response[neuron] = K_res(t, x_train[sample, neuron])
          for out_id in range(num_class):
            V[out_id, t] = Response*weight[:, out_id] + V_rest

        # Find the firing timing t_max
        for out_id in range(num_class):
          fire_time = np.where((V > threshold)[out_id, :])[0]
          if len(fire_time) > 0: # set firing time between 1 to 100
            max_state[0, out_id] = fire_time[0] + 1
            t_max[out_id] = fire_time[0] + 1
          else:
            max_state[0, out_id] = T
          
          if max_state[0, out_id] < T: # if output neuron fires
            V[out_id, t_max[out_id]:] = V[out_id, t_max[out_id]]*np.exp((time[t_max[out_id]:]-time[t_max[out_id]])/tou_m)

        pred_class = np.argmin(t_max)
        # Modify weight when error occurs: stochastic learning
        target = y_train[sample] # if target class != predict class, modify the weight
        if pred_class != target:
          for out_id in range(num_class):
            if out_id == target:
              if max_state[out_id] < threshold: # if the neuron supposed to fire did not fire
                for neuron in range(gauss_neurons):
                  if input_spike[sample, neuron] < t_max[out_id]: # for input neuron that contribute to the firing
                    # enhance the weight connection to the output neuron corresponding to target class
                    weight[neuron, out_id] += lr*K_res(t_max[out_id], input_spike[sample, out_id])

            elif out_id != target:
              if max_state[out_id] >= threshold: # if neuron shouldn't fired are erupted to fire
                for neuron in range(gauss_neurons):
                  if input_spike[sample, neuron] < t_max[out_id]:
                    # for input neuron contribute to the spike, inhibit the firing
                    weight[neuron, out_id] -= lr*K_res(t_max[out_id], input_spike[sample, out_id])

        # Even if predict class is correct, if the output neuron other than target class is evoked,
        # we should inhibit the firing of these neurons
        elif pred_class == target:
          for out_id in range(num_class):
            if out_id != target:
              if max_state[out_id] >= threshold:
                for neuron in range(gauss_neurons):
                  if input_spike[sample, neuron] < t_max[out_id]:
                    weight[neuron, out_id] -= lr*K_res(t_max[out_id], input_spike[sample, out_id])

      

train()       






ValueError: ignored